In [93]:
import pandas as pd
from collections import Counter
import nltk
import os
from gensim.models import KeyedVectors


In [62]:
metadata_table_long = pd.read_csv("../data/metadata_table_long.csv")


In [63]:
metadata_table_long.head(5)

,Author,Full title,In,Year,Place,Publisher/Printer,Era,Form/Genre,Discipline/Content,Original,...,Of interest to,Transkribus text available,Written by,Library and Signature,ids,id,date_min,date_max,filename,file_year
0,"Pardies, Ignace Gaston",A Latin Letter written to the Publisher April ...,Philosophical Transactions of the Royal Societ...,1672,London,Martyn,17th century,"Letter, Review",Physics,A Latin Letter(JSTOR),...,NaN,Yes,IT,NaN,[607720],607720,1672.0,1672.0,"Pardies,_Ignace_Gaston_-_A_Latin_Letter_contai...",1672.0
1,"Scheuchzer, Johann Jakob","Acarnania sive Relatio eorum, quae hactenus el...","ΟΥΡΕΣΙΦΟΙΤΗΣ (Ouresiphoites) Helveticus, 609–35",1723,Leiden,"van der Aa, Pieter",18th century,"Biography, Bibliography","Mathematics, Physics, Geography/Cartography, M...","Acarnania (1st edition), inMiscellanea Lipsien...",...,MK,Yes,MK,NaN,[605919],605919,1723.0,1723.0,"Scheuchzer,_Johann_Jakob_-_ΟΥΡΕΣΙΦΟΙΤΗΣ_(Oures...",1723.0
2,"Scheuchzer, Johann Jakob",ΟΥΡΕΣΙΦΟΙΤΗΣ Helveticus sive itinera per Helve...,NaN,1723,Leiden,"van der Aa, Pieter",18th century,"Report, Bibliography","Geography/Cartography, Meteorology/Earth scien...",ΟΥΡΕΣΙΦΟΙΤΗΣ (Ouresiphoites) Helveticus(e-rara...,...,MK,Yes,MK,NaN,[605919],605919,1723.0,1723.0,"Scheuchzer,_Johann_Jakob_-_ΟΥΡΕΣΙΦΟΙΤΗΣ_(Oures...",1723.0
3,"Addison, Joseph",Ad insignissimum virum dominum Thomam Burnettu...,"Examen poeticum duplex, sive, Musarum anglican...",1698,London,Richard Wellington I.,17th century,Panegyric poem,Meteorology/Earth sciences,Ad Burnettum sacrae theoriae telluris auctorem...,...,"MK, IT",Yes,MK,NaN,[769230],769230,1698.0,1698.0,Examen_poeticum_duplex__London_1698_pdf.txt,1698.0
4,"Addison, Joseph",Barometri descriptio,"Examen poeticum duplex, sive, Musarum anglican...",1698,London,Richard Wellington I.,17th century,Didactic poem,Physics,"Barometri descriptio, in:Examen poeticum duple...",...,NaN,Yes,NaN,NaN,[769230],769230,1698.0,1698.0,Examen_poeticum_duplex__London_1698_pdf.txt,1698.0


In [64]:
id = metadata_table_long["id"][1]

In [65]:
with open("../data/large_data/sents_lemmata/{}.txt".format(str(id)), "r") as f:
    sents_lemmata = [sent.strip().split() for sent in f.readlines()]

In [66]:
lemmata = [l for s in sents_lemmata for l in s]
lemmata[:20]

['ioh',
 'iac',
 'scheuchzeri',
 'itinera',
 'alpina',
 'plurimus',
 'tabula',
 'aeneus',
 'illustro',
 'lugud',
 'bat',
 'sumptibus',
 'petrus',
 'vander',
 'aa.',
 'ουρεσιφοιτης',
 'helveticus',
 'sive',
 'itinera',
 'per']

In [67]:
word_counts = dict(nltk.FreqDist(lemmata).most_common())

In [126]:
files_ready = os.listdir("../data/large_data/sents_lemmata")
len(files_ready)

553

In [98]:
%%time
files_ready = os.listdir("../data/large_data/sents_lemmata")
vocab_counter = Counter()
overview_dict = {}
for fn in files_ready:
    with open("../data/large_data/sents_lemmata/" + fn, "r") as f:
        sents_lemmata = [sent.strip().split() for sent in f.readlines()]
    sents_n = len(sents_lemmata)
    lemmata = [l for s in sents_lemmata for l in s]
    words_n = len(lemmata)
    overview_dict[fn.partition(".")[0]] = {"sents_n" : sents_n, "words_n" : words_n} 
    word_counts = dict(nltk.FreqDist(lemmata).most_common())
    vocab_counter.update(word_counts)

CPU times: user 37.9 s, sys: 953 ms, total: 38.9 s
Wall time: 42 s


In [99]:
sum(vocab_counter.values())

43135379

In [100]:
vocab_counter

Counter({'sum': 1106059,
         'in': 1009892,
         'qui': 896546,
         'hic': 380930,
         'is': 367355,
         'de': 356110,
         'que': 353692,
         'ad': 346037,
         'non': 343268,
         'ex': 280048,
         'ab': 258274,
         'atque': 211371,
         'cum': 209345,
         'ille': 209024,
         'ut': 203382,
         'à': 181800,
         'sed': 173923,
         'si': 168298,
         'ipse': 166071,
         'per': 163995,
         'alius': 163485,
         'omnis': 157359,
         'dico': 151053,
         'et': 147701,
         'se': 136464,
         'uel': 134465,
         'possum': 131941,
         'pars': 128654,
         'facio': 126241,
         'idem': 125841,
         'aut': 125139,
         'etiam': 123605,
         'vt': 122009,
         'habeo': 120844,
         'suus': 109980,
         'autem': 108246,
         'res': 106409,
         'greek.expression': 105917,
         '2': 105278,
         'quod': 103876,
         'enim':

In [117]:
N = 20
vocab_thres5 = [tup for tup in zip(vocab_counter.keys(), vocab_counter.values()) if tup[1] >= N]
sum([tup[1] for tup in vocab_thres5])

39957729

In [118]:
len(vocab_thres5)

79520

In [109]:
lasla = KeyedVectors.load_word2vec_format("../data/large_data/allLASLA-lemmi-fast-100-SKIP-win5-min5.vec")
operamayora = KeyedVectors.load_word2vec_format("../data/large_data/opera-maiora-lemmas_skip_100.vec")


In [119]:
lasla_vocab = lasla.index_to_key
len(lasla_vocab)

11327

In [121]:
noscemus_vocab_aboveN = [el[0] for el in vocab_thres5]

In [123]:
lasla_noscemus_overlap = list(set(lasla_vocab) & set(noscemus_vocab_aboveN)) 

In [124]:
len(lasla_noscemus_overlap)

8982

In [125]:
lasla_noscemus_overlap

['mutio',
 'appareo',
 'insero',
 'quemadmodum',
 'phoenice',
 'deprimo',
 'promissum',
 'nihilum',
 'sto',
 'melleus',
 'decus',
 'temperatus',
 'demens',
 'nuntio',
 'uiaticum',
 'procerus',
 'patrona',
 'grauis',
 'saturo',
 'impono',
 'sponsio',
 'memmius',
 'lente',
 'communitas',
 'absis',
 'serenitas',
 'abeo',
 'leuiter',
 'catillus',
 'siligineus',
 'debilitas',
 'significatio',
 'euomo',
 'persisto',
 'minutus',
 'prandium',
 'arctus',
 'cos',
 'delia',
 'piger',
 'amburo',
 'status',
 'irrigo',
 'bithynus',
 'uiduus',
 'pernosco',
 'religio',
 'destillo',
 'centuria',
 'insidiator',
 'agna',
 'corinthius',
 'qualiscumque',
 'paulisper',
 'austerus',
 'huc',
 'exedo',
 'cadus',
 'caelicola',
 'lego',
 'undecimus',
 'cohors',
 'niger',
 'effusus',
 'famulus',
 'alica',
 'ubi',
 'saturnus',
 'pusillum',
 'praetexo',
 'niualis',
 'nympha',
 'clientela',
 'annalis',
 'transeo',
 'arundo',
 'lautitia',
 'destinatus',
 'rescindo',
 'themistocles',
 'ara',
 'area',
 'immaturus',
 'p